In [1]:
!pip install scikit-learn==1.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 90.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import seaborn as sns

import os
from sklearn.impute import SimpleImputer
# machine learning tools
import h2o
from h2o.estimators import H2OGeneralizedLinearEstimator, H2ORandomForestEstimator, H2OGradientBoostingEstimator
from h2o.automl import H2OAutoML

# show files
!ls -l '/kaggle/input/brist1d'

total 295528
-rw-r--r-- 1 nobody nogroup       209 Sep 24 01:28 activities.txt
-rw-r--r-- 1 nobody nogroup     50703 Sep 24 01:28 sample_submission.csv
-rw-r--r-- 1 nobody nogroup   5965689 Sep 24 01:28 test.csv
-rw-r--r-- 1 nobody nogroup 296592806 Sep 24 01:28 train.csv


In [3]:
# configs
pd.set_option('display.max_columns', None) # we want to display all columns in this notebook
pd.set_option('display.max_rows', 100) # increase rows to be displayed
pd.set_option('display.max_colwidth', None) # show full cell contents

# random seed
my_random_seed = 111

# aesthetics
default_color_1 = 'darkblue'
default_color_2 = 'darkgreen'
default_color_3 = 'darkred'

import warnings
warnings.filterwarnings('ignore')

In [4]:
# load data
t1 = time.time()

train = pd.read_csv('/kaggle/input/brist1d/train.csv', low_memory=False) 
test = pd.read_csv('/kaggle/input/brist1d/test.csv', low_memory=False)
sub = pd.read_csv('/kaggle/input/brist1d/sample_submission.csv', low_memory=False)
# low_memory=False : 대용량 데이터 불러올 때 메모리 에러 무시
t2 = time.time()
print('Elapsed time [s]:', np.round(t2-t1,4))

Elapsed time [s]: 18.2938


In [5]:
import numpy as np
from prettytable import PrettyTable
table = PrettyTable()

target = 'bg+1:00'
# table.field_names = ['Feature', 'Data Type', 'Train Missing %', 'Test Missing %']
# for column in train.columns:
#     data_type = str(train[column].dtype)
#     non_null_count_train= np.round(100-train[column].count()/train.shape[0]*100,1)
#     if column!=target:
#         non_null_count_test = np.round(100-test[column].count()/test.shape[0]*100,1)
#     else:
#         non_null_count_test="NA"
#     table.add_row([column, data_type, non_null_count_train,non_null_count_test])
# print(table)

In [6]:
# import missingno as msno

# msno.matrix(train)
# msno.matrix(test)

In [7]:
# # plot target distribution
# plt.figure(figsize=(12,4))
# train[target].plot(kind='hist', bins=50,
#                       color=default_color_3)
# plt.title(target)
# plt.grid()
# plt.show()
# # basic stats
# print(train[target].describe())
# # 타겟이 편향되어 있음 > log 변환????

In [8]:
# train['log_target'] = np.log(train[target])

# # 로그 변환된 타겟 변수의 분포 시각화
# plt.figure(figsize=(10, 6))
# sns.histplot(train['log_target'], kde=True, bins=30,color=default_color_3)
# plt.xlabel('Log Transformed bg+1:00 (Target Variable)')
# plt.ylabel('Frequency')
# plt.title('Distribution of Log Transformed Target Variable (bg+1:00)')
# plt.show()

In [9]:
# # 타겟 변수 설정
# target_column = 'bg+1:00'

# # 숫자형 데이터만 선택
# numeric_data = train.select_dtypes(include=['float64', 'int64'])

# # 타겟 변수와 나머지 숫자형 변수 간의 상관관계 계산
# if target_column in numeric_data.columns:
#     correlation_with_target = numeric_data.corr()[target_column].drop(target_column)
# else:
#     print(f"타겟 변수 '{target_column}'이 데이터에 존재하지 않습니다.")
#     correlation_with_target = None
# print(correlation_with_target.index)
# print(correlation_with_target.values)
# # 상관관계 시각화
# if correlation_with_target is not None:
#     # 열 이름의 접두어를 기반으로 색상 지정
#     unique_prefixes = correlation_with_target.index.str.extract(r'(^[a-zA-Z_]+)', expand=False).unique()
#     color_map = {prefix: color for prefix, color in zip(unique_prefixes, sns.color_palette('Set2', len(unique_prefixes)))}
#     colors = [color_map[col.split('-')[0]] for col in correlation_with_target.index]

#     plt.figure(figsize=(12, 6))
#     sns.barplot(x=correlation_with_target.index, y=correlation_with_target.values, palette=colors)
#     plt.xticks(rotation=45, ha='right')
#     plt.xlabel('Features')
#     plt.ylabel('Correlation with Target (bg+1:00)')
#     plt.title('Correlation of Features with Target Variable (bg+1:00)')

#     # 범례 추가
#     from matplotlib.patches import Patch
#     legend_labels = [Patch(color=color, label=prefix) for prefix, color in color_map.items()]
#     plt.legend(handles=legend_labels, loc='upper right')

#     plt.tight_layout()
#     plt.show()

In [10]:
# # date conversion
# train.time = pd.to_datetime(train.time, format='%H:%M:%S')
# test.time = pd.to_datetime(test.time, format='%H:%M:%S')
# # extract hour
# train['hour'] = train.time.apply(lambda x : x.hour)
# test['hour'] = test.time.apply(lambda x : x.hour)


In [11]:
# # plot target distribution by hour
# plt.figure(figsize=(14,6))
# sns.violinplot(data=train, x='hour', y=target)
# plt.title('Target by hour')
# plt.grid()
# plt.show()

시간과 타겟은 무관하다는 사실을 알 수 있다.

In [12]:
# # plot target distribution by patient
# plt.figure(figsize=(12,5))
# sns.violinplot(data=train, x='p_num', y=target)
# plt.title('Target by patient')
# plt.grid()
# plt.show()

환자 별 혈당이 다르다는 사실을 알 수 있다.

In [13]:
# from pandas.tseries.offsets import Minute # -시:분을( ex) bg-5:55) -355분으로 변환
# import re
# df = train
# df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')
# bg_cols = [col for col in df.columns if col.startswith('bg-')]

# time_offsets = {}
# for col in bg_cols:
#     match = re.search(r'bg-(\d+):(\d+)', col)
#     print('match :', match)
#     if match:
#         hours, minutes = int(match.group(1)), int(match.group(2))
#         total_minutes = hours * 60 + minutes  
#         print(f'total_minutes = {total_minutes}')
#         offset = -(total_minutes)  
#         print(f'offset = {offset}')
#         time_offsets[col] = offset


In [14]:
# train[train['p_num'] == 'p01'][['p_num','time']]

In [15]:
# time_series_list = []

# for col, offset in time_offsets.items():
#     temp_df = train[['p_num', 'time']].copy()  
#     temp_df['measurement_time'] = train['time'] + Minute(offset)
#     # print(f"train['time'] = {train['time']} \n Minute(offset) = {Minute(offset)} \n measure = {temp_df['measurement_time']}")

#     temp_df['value'] = train[col] 
#     # print(f'train[{col}] = \n {train[col]}')
#     temp_df['original_time'] = train['time'] 
#     temp_df['bg_column'] = col  #
#     time_series_list.append(temp_df)
# # print('time_series_list', time_series_list)
# time_series_data = pd.concat(time_series_list, ignore_index=True).dropna(subset=['value']) 
# # 각 시간의 offset만큼 뺀 데이터프레임들을 하나로 합친다.
# time_series_data

In [16]:
# time_series_data.groupby(['p_num', 'hour'])['value'].mean()
# time_series_data.groupby(['p_num', 'hour'])['value'].mean().unstack()

In [17]:
# time_series_data['measurement_time'] = pd.to_datetime(time_series_data['measurement_time'])
# time_series_data['hour'] = time_series_data['measurement_time'].dt.hour
# hourly_avg_by_pnum = time_series_data.groupby(['p_num', 'hour'])['value'].mean().unstack()
# # print('hourly', hourly_avg_by_pnum)
# # print(time_series_data)
# plt.figure(figsize=(12, 8))
# for p_num in hourly_avg_by_pnum.index:
#     plt.plot(hourly_avg_by_pnum.columns, hourly_avg_by_pnum.loc[p_num], marker='o', label=f'p_num: {p_num}')

# plt.title('Average Value by Hour for Each p_num (24-hour cycle)')
# plt.xlabel('Hour of the Day')
# plt.ylabel('Average Value')
# plt.grid(True)
# plt.xticks(range(24))  
# plt.legend(title='p_num')
# plt.show()

In [18]:
# numeric_data_train = train.select_dtypes(include=['float64', 'int64']).drop(['bg+1:00', 'log_target'], axis=1)
# numeric_data_test = test.select_dtypes(include=['float64', 'int64'])
# filled_train = train.copy()
# filled_test = test.copy()

In [19]:
# imputer = SimpleImputer()

# filled_train[numeric_data_train.columns] = imputer.fit_transform(numeric_data_train)
# filled_test[numeric_data_test.columns] = imputer.transform(numeric_data_test)
# filled_train

In [20]:
# df_train = train.copy()
# df_test = test.copy()
# df_train['time_min'] = train.hour * 60 + train.time.apply(lambda x : x.minute)
# df_test['time_min'] = test.hour * 60 + test.time.apply(lambda x : x.minute)
# count_train = df_train.p_num.value_counts().sort_index()
# for x in count_train.index:
#     df_sel = df_train[df_train.p_num==x]
#     # print(df_sel.filter(regex = '^step').mean(axis=1))
#     # print('step null', df_sel.filter(regex = '^step').mean(axis=1).isnull().sum())
#     # print(df_sel.filter(regex = '^cals').mean(axis=1))
#     # print('cals null', df_sel.filter(regex = '^cals').mean(axis=1).isnull().sum())
#     print(df_sel.filter(regex = '^carbs').mean(axis=1))
#     print('carbs null', df_sel.filter(regex = '^carbs').mean(axis=1).isnull().sum())
#     # print(df_sel.time_min.shape)
#     plt.figure(figsize=(14,4))
#     # plt.scatter(df_sel.time_min, df_sel.filter(regex = '^step').mean(axis=1), color=default_color_3, 
#     #             alpha=0.2, s=5)
#     # plt.scatter(df_sel.time_min, df_sel.filter(regex = '^cals').mean(axis=1), color=default_color_3, 
#     #             alpha=0.2, s=5)
#     plt.scatter(df_sel.time_min, df_sel.filter(regex = '^carbs').mean(axis=1), color=default_color_3, 
#                 alpha=0.2, s=5)
#     plt.title(x)
#     # plt.ylabel('steps')
#     # plt.ylabel('carls')
#     plt.ylabel('carbs')
#     plt.xlabel('time_min')
#     plt.grid()
#     plt.show()

In [21]:
# hr_cols = filled_train.filter(regex='^hr', axis=1).columns

# filled_train= filled_train.drop(hr_cols, axis=1)
# filled_train

In [22]:
# hr_cols_test = filled_test.filter(regex='^hr', axis=1).columns

# filled_test= filled_test.drop(hr_cols_test, axis=1)
# filled_test

In [23]:
# filled_train = filled_train.drop('bg+1:00', axis=1)

In [24]:
# train = pd.read_csv('/kaggle/input/autogluondata/train_data.csv', low_memory=False)
# test = pd.read_csv('/kaggle/input/autogluondata/test_data.csv', low_memory=False)

In [25]:
!pip install --upgrade pip setuptools wheel
!pip install onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.43.0
    Uninstalling wheel-0.43.0:
      Successfully uninstalled wheel-0.43.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 70.0.0
    Uninstalling setuptools-70.0.0:
      Successfully uninstalled setuptools-70.0.0
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 24.9.0 requires packaging>=23.0, but you have packaging 21.3 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0

In [26]:
!pip install autogluon==0.8.2


  Preparing metadata (setup.py) ... - \ | error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [61 lines of output]
      /opt/conda/lib/python3.10/site-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
      [12/07/24 05:59:25] WARNING  toml section missing        ]8;id=684582;file:///opt/conda/lib/python3.10/site-packages/setuptools_scm/_integration/pyproject_reading.py\pyproject_reading.py]8;;\:]8;id=755269;file:///opt/conda

In [27]:
!pip install autogluon.tabular


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 101.1 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.23
    Uninstalling botocore-1.35.23:
      Successfully uninstalled botocore-1.35.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.15.1 requires botocore<1.35.24,>=1.35.16, but you have botocore 1.29.165 which is incompatible.


In [28]:
# len(filled_train.columns), len(filled_test.columns)

In [29]:
print(train.shape)
print(test.shape)
# train과 test 데이터를 합침 (열 이름이 동일하다는 가정 하에)
all_data = pd.concat([train, test], axis=0, ignore_index=True)

# 합쳐진 데이터 확인
print("Train과 Test 데이터를 결합한 결과:")
print(all_data.head())
print("\n결합된 데이터의 크기:", all_data.shape)

(177024, 508)
(3644, 507)
Train과 Test 데이터를 결합한 결과:
      id p_num      time  bg-5:55  bg-5:50  bg-5:45  bg-5:40  bg-5:35  \
0  p01_0   p01  06:10:00      NaN      NaN      9.6      NaN      NaN   
1  p01_1   p01  06:25:00      NaN      NaN      9.7      NaN      NaN   
2  p01_2   p01  06:40:00      NaN      NaN      9.2      NaN      NaN   
3  p01_3   p01  06:55:00      NaN      NaN      8.7      NaN      NaN   
4  p01_4   p01  07:10:00      NaN      NaN      8.4      NaN      NaN   

   bg-5:30  bg-5:25  bg-5:20  bg-5:15  bg-5:10  bg-5:05  bg-5:00  bg-4:55  \
0      9.7      NaN      NaN      9.2      NaN      NaN      8.7      NaN   
1      9.2      NaN      NaN      8.7      NaN      NaN      8.4      NaN   
2      8.7      NaN      NaN      8.4      NaN      NaN      8.1      NaN   
3      8.4      NaN      NaN      8.1      NaN      NaN      8.3      NaN   
4      8.1      NaN      NaN      8.3      NaN      NaN      9.6      NaN   

   bg-4:50  bg-4:45  bg-4:40  bg-4:35  bg-4:30 

In [30]:
# "bg-"로 시작하는 혈당 관련 열 이름 필터링, bg-0:00 및 bg-5:55 제외 -> 이전, 이후값 영향
blood_sugar_columns_exclude_edges = [col for col in all_data.columns if col.startswith('bg-') and col not in ['bg-5:55', 'bg-0:00']]

# 선형 보간법 적용
all_data[blood_sugar_columns_exclude_edges] = all_data[blood_sugar_columns_exclude_edges].interpolate(method='linear', axis=1)

# 처리 전후 결측치 확인
print("보간법 적용 후 결측치 개수:")
print(all_data[blood_sugar_columns_exclude_edges].isna().sum())

# 결과 확인
print("bg-0:00 제외 후 선형 보간법 적용된 데이터:")
print(all_data[blood_sugar_columns_exclude_edges].head())
print(all_data[blood_sugar_columns_exclude_edges].isna().sum())




보간법 적용 후 결측치 개수:
bg-5:50    19074
bg-5:45     4242
bg-5:40     3580
bg-5:35     3456
bg-5:30     3268
bg-5:25     3200
bg-5:20     3125
bg-5:15     2966
bg-5:10     2899
bg-5:05     2827
bg-5:00     2678
bg-4:55     2614
bg-4:50     2545
bg-4:45     2406
bg-4:40     2344
bg-4:35     2276
bg-4:30     2144
bg-4:25     2081
bg-4:20     2013
bg-4:15     1895
bg-4:10     1834
bg-4:05     1769
bg-4:00     1655
bg-3:55     1590
bg-3:50     1525
bg-3:45     1423
bg-3:40     1374
bg-3:35     1328
bg-3:30     1246
bg-3:25     1209
bg-3:20     1170
bg-3:15     1095
bg-3:10     1062
bg-3:05     1028
bg-3:00      962
bg-2:55      930
bg-2:50      898
bg-2:45      838
bg-2:40      808
bg-2:35      778
bg-2:30      722
bg-2:25      696
bg-2:20      670
bg-2:15      618
bg-2:10      595
bg-2:05      572
bg-2:00      525
bg-1:55      503
bg-1:50      480
bg-1:45      442
bg-1:40      420
bg-1:35      398
bg-1:30      360
bg-1:25      337
bg-1:20      314
bg-1:15      275
bg-1:10      256
bg-1:05      2

In [31]:
print(all_data[blood_sugar_columns_exclude_edges].isna().sum().sum())
# 안채워진 값들은 앞 뒤 값들 이용해서 채우기
all_data[blood_sugar_columns_exclude_edges]  = all_data[blood_sugar_columns_exclude_edges].bfill(axis = 1)
all_data[blood_sugar_columns_exclude_edges]  = all_data[blood_sugar_columns_exclude_edges].ffill(axis = 1)

105826


In [32]:
# 'bg-5:55' 결측치를 'bg-5:50' 값으로 채우기
all_data['bg-5:55'] = all_data['bg-5:55'].fillna(all_data['bg-5:50'])

In [33]:
# 'bg-0:00' 결측치를 'bg-0:05' 값으로 채우기
all_data['bg-0:00'] = all_data['bg-0:00'].fillna(all_data['bg-0:05'])
all_data['bg-0:00'].isna().sum()

0

In [34]:
# 혈당 열 선택
blood_sugar_columns = [col for col in all_data.columns if col.startswith('bg-')]
print(all_data[blood_sugar_columns].isna().sum().sum())

0


In [35]:
# 인슐린 열
insulin_columns = [col for col in all_data.columns if col.startswith('insulin-')]
print(all_data[insulin_columns].isna().sum().sum())

688995


In [36]:
## 인슐린도 혈당과 같이 진행
insulin_columns_exclude_edges = [col for col in all_data.columns if col.startswith('insulin-') and col not in ['insulin-5:55', 'insulin-0:00']]
print(all_data[insulin_columns_exclude_edges].isna().sum().sum())
# 선형 보간법 적용
all_data[insulin_columns_exclude_edges] = all_data[blood_sugar_columns_exclude_edges].interpolate(method='linear', axis=1)
print(all_data[insulin_columns_exclude_edges].isna().sum().sum())

669857
0


In [37]:

# 안채워진 값들은 앞 뒤 값들 이용해서 채우기
all_data[insulin_columns_exclude_edges]  = all_data[insulin_columns_exclude_edges].bfill(axis = 1)
all_data[insulin_columns_exclude_edges]  = all_data[insulin_columns_exclude_edges].ffill(axis = 1)
print(all_data[insulin_columns_exclude_edges].isna().sum().sum())

0


In [38]:
# 'insulin-5:55' 결측치를 'insulin-5:50' 값으로 채우기
all_data['insulin-5:55'] = all_data['insulin-5:55'].fillna(all_data['insulin-5:50'])
all_data['insulin-5:55'].isna().sum()
# 'insulin-0:00' 결측치를 'insulin-0:05' 값으로 채우기
all_data['insulin-0:00'] = all_data['insulin-0:00'].fillna(all_data['insulin-0:05'])
all_data['insulin-0:00'].isna().sum()

0

In [39]:
## 탄수화물 섭취량은 섭취를 안했을때 0이라 생각해서 0으로 결측치 채우기
# "carbs-"로 시작하는 모든 열 이름 필터링
carbs_columns = [col for col in all_data.columns if col.startswith('carbs-')]

# 해당 열들의 결측치를 모두 0으로 채우기
all_data[carbs_columns] = all_data[carbs_columns].fillna(0)

# 결과 확인
print("결측치를 0으로 채운 'carbs-' 열들 상위 5개:")
print(all_data[carbs_columns].head())


결측치를 0으로 채운 'carbs-' 열들 상위 5개:
   carbs-5:55  carbs-5:50  carbs-5:45  carbs-5:40  carbs-5:35  carbs-5:30  \
0         0.0         0.0         0.0         0.0         0.0         0.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         0.0         0.0         0.0         0.0         0.0   
3         0.0         0.0         0.0         0.0         0.0         0.0   
4         0.0         0.0         0.0         0.0         0.0         0.0   

   carbs-5:25  carbs-5:20  carbs-5:15  carbs-5:10  carbs-5:05  carbs-5:00  \
0         0.0         0.0         0.0         0.0         0.0         0.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         0.0         0.0         0.0         0.0         0.0   
3         0.0         0.0         0.0         0.0         0.0         0.0   
4         0.0         0.0         0.0         0.0         0.0         0.0   

   carbs-4:55  carbs-4:50  carbs-4:45  carb

In [40]:
# 심장박동수도 혈당과 같이 처리하기
# 심박수 열
hr_columns = [col for col in all_data.columns if col.startswith('hr-')]
print(all_data[hr_columns].isna().sum().sum())

3815694


In [41]:
# "hr-"로 시작하는 열에서 시작과 끝을 제외한 열 필터링
hr_columns_exclude_edges = [col for col in all_data.columns if col.startswith('hr-') and col not in ['hr-5:55', 'hr-0:00']]

# 결측치 개수 확인
print("선형 보간 전 결측치 합계:")
print(all_data[hr_columns_exclude_edges].isna().sum().sum())

# 선형 보간법 적용
all_data[hr_columns_exclude_edges] = all_data[hr_columns_exclude_edges].interpolate(method='linear', axis=1)

# 보간 후 결측치 개수 확인
print("선형 보간 후 결측치 합계:")
print(all_data[hr_columns_exclude_edges].isna().sum().sum())

선형 보간 전 결측치 합계:
3709792
선형 보간 후 결측치 합계:
2657364


In [42]:
# 안채워진 값들은 앞 뒤 값들 이용해서 채우기
all_data[hr_columns_exclude_edges]  = all_data[hr_columns_exclude_edges].bfill(axis = 1)
all_data[hr_columns_exclude_edges]  = all_data[hr_columns_exclude_edges].ffill(axis = 1)
print(all_data[hr_columns_exclude_edges].isna().sum().sum())

2022720


In [43]:
## 6시간 동안 심박수관련 열이 하나도 없는 경우 존재, 심박수의 상관관계가 크지 않음 -> 심박수 관련 열은 삭제하기로 결정

print(all_data.shape)
# "hr-"로 시작하는 모든 열 필터링
hr_columns = [col for col in all_data.columns if col.startswith('hr-')]

# 해당 열 삭제
all_data = all_data.drop(columns=hr_columns)

# 삭제 결과 확인
print(f"삭제된 'hr-' 관련 열 개수: {len(hr_columns)}")
print(f"현재 데이터프레임의 열 개수: {all_data.shape[1]}")

print(all_data.shape)

(180668, 508)
삭제된 'hr-' 관련 열 개수: 72
현재 데이터프레임의 열 개수: 436
(180668, 436)


In [44]:
## 걸음수는 걷지 않을때 0이라 생각해서 0으로 결측치 채우기
# "steps-"로 시작하는 모든 열 이름 필터링
steps_columns = [col for col in all_data.columns if col.startswith('steps-')]

# 해당 열들의 결측치를 모두 0으로 채우기
all_data[steps_columns] = all_data[steps_columns].fillna(0)

# 결과 확인
print("결측치를 0으로 채운 'carbs-' 열들 상위 5개:")
print(all_data[steps_columns].head())
print(all_data[steps_columns].isna().sum().sum())

결측치를 0으로 채운 'carbs-' 열들 상위 5개:
   steps-5:55  steps-5:50  steps-5:45  steps-5:40  steps-5:35  steps-5:30  \
0         0.0         0.0         0.0         0.0         0.0         0.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         0.0         0.0         0.0         0.0         0.0   
3         0.0         0.0         0.0         0.0         0.0         0.0   
4         0.0         0.0         0.0         0.0         0.0         0.0   

   steps-5:25  steps-5:20  steps-5:15  steps-5:10  steps-5:05  steps-5:00  \
0         0.0         0.0         0.0         0.0         0.0         0.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         0.0         0.0         0.0         0.0         0.0   
3         0.0         0.0         0.0         0.0         0.0         0.0   
4         0.0         0.0         0.0         0.0         0.0         0.0   

   steps-4:55  steps-4:50  steps-4:45  step

In [45]:
## 칼로리 또한 소모하지 않을때 0이라 생각해서 0으로 결측치 채우기
# "cals-"로 시작하는 모든 열 이름 필터링
cals_columns = [col for col in all_data.columns if col.startswith('cals-')]

# 해당 열들의 결측치를 모두 0으로 채우기
all_data[cals_columns] = all_data[cals_columns].fillna(0)

# 결과 확인
print("결측치를 0으로 채운 'carbs-' 열들 상위 5개:")
print(all_data[cals_columns].head())
print(all_data[cals_columns].isna().sum().sum())

결측치를 0으로 채운 'carbs-' 열들 상위 5개:
   cals-5:55  cals-5:50  cals-5:45  cals-5:40  cals-5:35  cals-5:30  \
0        0.0        0.0        0.0        0.0        0.0        0.0   
1        0.0        0.0        0.0        0.0        0.0        0.0   
2        0.0        0.0        0.0        0.0        0.0        0.0   
3        0.0        0.0        0.0        0.0        0.0        0.0   
4        0.0        0.0        0.0        0.0        0.0        0.0   

   cals-5:25  cals-5:20  cals-5:15  cals-5:10  cals-5:05  cals-5:00  \
0        0.0        0.0        0.0        0.0        0.0        0.0   
1        0.0        0.0        0.0        0.0        0.0        0.0   
2        0.0        0.0        0.0        0.0        0.0        0.0   
3        0.0        0.0        0.0        0.0        0.0        0.0   
4        0.0        0.0        0.0        0.0        0.0        0.0   

   cals-4:55  cals-4:50  cals-4:45  cals-4:40  cals-4:35  cals-4:30  \
0        0.0        0.0        0.0        0.

In [46]:

# "activity-"로 시작하는 모든 열 이름 필터링
activity_columns = [col for col in all_data.columns if col.startswith('activity-')]

In [47]:
# 모든 "activity" 열의 고유 값 출력
unique_values = all_data[activity_columns].stack().unique()
print("모든 'activity' 열의 고유 값:")
print(unique_values)

모든 'activity' 열의 고유 값:
['Walk' 'Indoor climbing' 'Yoga' 'Zumba' 'HIIT' 'Dancing' 'Swim'
 'Outdoor Bike' 'Aerobic Workout' 'Sport' 'Walking' 'Running' 'Swimming'
 'Run' 'Weights' 'Workout' 'Tennis' 'Strength training' 'Stairclimber'
 'Spinning' 'Hike' 'Bike' 'Cycling' 'CoreTraining']


In [48]:
# 활동 데이터에서 "swiming"을 "swim"으로, "walking"을 "walk"로 변환
# 변환 함수 적용
for col in activity_columns:
    all_data[col] = all_data[col].replace({
        "Swiming": "Swim",
        "Walking": "Walk",
        "Running": "Run"
    })


In [49]:
# 순서형 라벨링을 위한 활동 목록 재정의 (격한 정도 순서대로)
activity_order = [
    "HIIT","Run", "Outdoor Bike", "Cycling", "Spinning",
    "Stairclimber", "Strength training", "CoreTraining", "Indoor climbing", 
    "Swim", "Aerobic Workout", "Tennis", "Sport", "Hike", 
    "Weights", "Dancing", "Zumba", "Bike", "Workout","Walk", "Yoga"
]

# "activity"로 시작하는 모든 열 필터링
activity_columns = [col for col in all_data.columns if col.startswith('activity')]

# 순서형 라벨을 모든 "activity" 열에 적용
for col in activity_columns:
    all_data[col] = all_data[col].apply(
        lambda x: len(activity_order) - activity_order.index(x) if x in activity_order else 0
    )

# 결과 확인
print("결측치 0 처리 및 격할수록 높은 값으로 라벨 인코딩 적용 후 상위 5개:")
print(all_data[activity_columns].head())


결측치 0 처리 및 격할수록 높은 값으로 라벨 인코딩 적용 후 상위 5개:
   activity-5:55  activity-5:50  activity-5:45  activity-5:40  activity-5:35  \
0              0              0              0              0              0   
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              0              0              0              0   
4              0              0              0              0              0   

   activity-5:30  activity-5:25  activity-5:20  activity-5:15  activity-5:10  \
0              0              0              0              0              0   
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              0              0              0              0   
4              0              0              0              0              0 

In [50]:
# "bg+1:00" 열을 제외한 데이터에서 결측치 개수 계산
missing_values_count = all_data.drop(columns=['bg+1:00']).isna().sum().sum()

# 결과 출력
print(f"'bg+1:00' 열을 제외한 결측치 개수: {missing_values_count}")

'bg+1:00' 열을 제외한 결측치 개수: 0


In [51]:
# 결측치가 있는 열만 필터링하여 출력
missing_columns = all_data.columns[all_data.isna().any()].tolist()

# 결측치가 있는 열 출력
print("결측치가 있는 열:")
print(missing_columns)


결측치가 있는 열:
['bg+1:00']


In [52]:
# 데이터의 크기를 바탕으로 train과 test 다시 분리
train_rows = 177024  # train 데이터 행 개수
test_rows = 3644  # test 데이터 행 개수

# train 데이터와 test 데이터 분리
train_data = all_data.iloc[:train_rows, :]
test_data = all_data.iloc[train_rows:train_rows + test_rows, :]

# 분리된 데이터 크기 확인
print(f"Train 데이터 크기: {train_data.shape}")
print(f"Test 데이터 크기: {test_data.shape}")

Train 데이터 크기: (177024, 436)
Test 데이터 크기: (3644, 436)


In [53]:
print(train.head())
print(train_data.head())
print(train.tail())
print(train_data.tail())

      id p_num      time  bg-5:55  bg-5:50  bg-5:45  bg-5:40  bg-5:35  \
0  p01_0   p01  06:10:00      NaN      NaN      9.6      NaN      NaN   
1  p01_1   p01  06:25:00      NaN      NaN      9.7      NaN      NaN   
2  p01_2   p01  06:40:00      NaN      NaN      9.2      NaN      NaN   
3  p01_3   p01  06:55:00      NaN      NaN      8.7      NaN      NaN   
4  p01_4   p01  07:10:00      NaN      NaN      8.4      NaN      NaN   

   bg-5:30  bg-5:25  bg-5:20  bg-5:15  bg-5:10  bg-5:05  bg-5:00  bg-4:55  \
0      9.7      NaN      NaN      9.2      NaN      NaN      8.7      NaN   
1      9.2      NaN      NaN      8.7      NaN      NaN      8.4      NaN   
2      8.7      NaN      NaN      8.4      NaN      NaN      8.1      NaN   
3      8.4      NaN      NaN      8.1      NaN      NaN      8.3      NaN   
4      8.1      NaN      NaN      8.3      NaN      NaN      9.6      NaN   

   bg-4:50  bg-4:45  bg-4:40  bg-4:35  bg-4:30  bg-4:25  bg-4:20  bg-4:15  \
0      NaN      8.4  

In [54]:
print(test.head())
print(test_data.head())
print(test.tail())
print(test_data.tail())

         id p_num      time  bg-5:55  bg-5:50  bg-5:45  bg-5:40  bg-5:35  \
0  p01_8459   p01  06:45:00      NaN      9.2      NaN      NaN     10.2   
1  p01_8460   p01  11:25:00      NaN      NaN      9.9      NaN      NaN   
2  p01_8461   p01  14:45:00      NaN      5.5      NaN      NaN      5.5   
3  p01_8462   p01  04:30:00      NaN      3.4      NaN      NaN      3.9   
4  p01_8463   p01  04:20:00      NaN      NaN      8.3      NaN      NaN   

   bg-5:30  bg-5:25  bg-5:20  bg-5:15  bg-5:10  bg-5:05  bg-5:00  bg-4:55  \
0      NaN      NaN     10.3      NaN      NaN     10.2      NaN      NaN   
1      9.4      NaN      NaN      9.1      NaN      NaN      8.3      NaN   
2      NaN      NaN      5.2      NaN      NaN      5.2      NaN      NaN   
3      NaN      NaN      4.7      NaN      NaN      4.1      NaN      NaN   
4     10.0      NaN      NaN     12.2      NaN      NaN     12.8      NaN   

   bg-4:50  bg-4:45  bg-4:40  bg-4:35  bg-4:30  bg-4:25  bg-4:20  bg-4:15  \
0  

In [55]:
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

print("Train 데이터와 Test 데이터가 각각 'train_data.csv'와 'test_data.csv'로 저장되었습니다.")

Train 데이터와 Test 데이터가 각각 'train_data.csv'와 'test_data.csv'로 저장되었습니다.


In [56]:
test_data.drop('bg+1:00', axis=1)

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,bg-5:15,bg-5:10,bg-5:05,bg-5:00,bg-4:55,bg-4:50,bg-4:45,bg-4:40,bg-4:35,bg-4:30,bg-4:25,bg-4:20,bg-4:15,bg-4:10,bg-4:05,bg-4:00,bg-3:55,bg-3:50,bg-3:45,bg-3:40,bg-3:35,bg-3:30,bg-3:25,bg-3:20,bg-3:15,bg-3:10,bg-3:05,bg-3:00,bg-2:55,bg-2:50,bg-2:45,bg-2:40,bg-2:35,bg-2:30,bg-2:25,bg-2:20,bg-2:15,bg-2:10,bg-2:05,bg-2:00,bg-1:55,bg-1:50,bg-1:45,bg-1:40,bg-1:35,bg-1:30,bg-1:25,bg-1:20,bg-1:15,bg-1:10,bg-1:05,bg-1:00,bg-0:55,bg-0:50,bg-0:45,bg-0:40,bg-0:35,bg-0:30,bg-0:25,bg-0:20,bg-0:15,bg-0:10,bg-0:05,bg-0:00,insulin-5:55,insulin-5:50,insulin-5:45,insulin-5:40,insulin-5:35,insulin-5:30,insulin-5:25,insulin-5:20,insulin-5:15,insulin-5:10,insulin-5:05,insulin-5:00,insulin-4:55,insulin-4:50,insulin-4:45,insulin-4:40,insulin-4:35,insulin-4:30,insulin-4:25,insulin-4:20,insulin-4:15,insulin-4:10,insulin-4:05,insulin-4:00,insulin-3:55,insulin-3:50,insulin-3:45,insulin-3:40,insulin-3:35,insulin-3:30,insulin-3:25,insulin-3:20,insulin-3:15,insulin-3:10,insulin-3:05,insulin-3:00,insulin-2:55,insulin-2:50,insulin-2:45,insulin-2:40,insulin-2:35,insulin-2:30,insulin-2:25,insulin-2:20,insulin-2:15,insulin-2:10,insulin-2:05,insulin-2:00,insulin-1:55,insulin-1:50,insulin-1:45,insulin-1:40,insulin-1:35,insulin-1:30,insulin-1:25,insulin-1:20,insulin-1:15,insulin-1:10,insulin-1:05,insulin-1:00,insulin-0:55,insulin-0:50,insulin-0:45,insulin-0:40,insulin-0:35,insulin-0:30,insulin-0:25,insulin-0:20,insulin-0:15,insulin-0:10,insulin-0:05,insulin-0:00,carbs-5:55,carbs-5:50,carbs-5:45,carbs-5:40,carbs-5:35,carbs-5:30,carbs-5:25,carbs-5:20,carbs-5:15,carbs-5:10,carbs-5:05,carbs-5:00,carbs-4:55,carbs-4:50,carbs-4:45,carbs-4:40,carbs-4:35,carbs-4:30,carbs-4:25,carbs-4:20,carbs-4:15,carbs-4:10,carbs-4:05,carbs-4:00,carbs-3:55,carbs-3:50,carbs-3:45,carbs-3:40,carbs-3:35,carbs-3:30,carbs-3:25,carbs-3:20,carbs-3:15,carbs-3:10,carbs-3:05,carbs-3:00,carbs-2:55,carbs-2:50,carbs-2:45,carbs-2:40,carbs-2:35,carbs-2:30,carbs-2:25,carbs-2:20,carbs-2:15,carbs-2:10,carbs-2:05,carbs-2:00,carbs-1:55,carbs-1:50,carbs-1:45,carbs-1:40,carbs-1:35,carbs-1:30,carbs-1:25,carbs-1:20,carbs-1:15,carbs-1:10,carbs-1:05,carbs-1:00,carbs-0:55,carbs-0:50,carbs-0:45,carbs-0:40,carbs-0:35,carbs-0:30,carbs-0:25,carbs-0:20,carbs-0:15,carbs-0:10,carbs-0:05,carbs-0:00,steps-5:55,steps-5:50,steps-5:45,steps-5:40,steps-5:35,steps-5:30,steps-5:25,steps-5:20,steps-5:15,steps-5:10,steps-5:05,steps-5:00,steps-4:55,steps-4:50,steps-4:45,steps-4:40,steps-4:35,steps-4:30,steps-4:25,steps-4:20,steps-4:15,steps-4:10,steps-4:05,steps-4:00,steps-3:55,steps-3:50,steps-3:45,steps-3:40,steps-3:35,steps-3:30,steps-3:25,steps-3:20,steps-3:15,steps-3:10,steps-3:05,steps-3:00,steps-2:55,steps-2:50,steps-2:45,steps-2:40,steps-2:35,steps-2:30,steps-2:25,steps-2:20,steps-2:15,steps-2:10,steps-2:05,steps-2:00,steps-1:55,steps-1:50,steps-1:45,steps-1:40,steps-1:35,steps-1:30,steps-1:25,steps-1:20,steps-1:15,steps-1:10,steps-1:05,steps-1:00,steps-0:55,steps-0:50,steps-0:45,steps-0:40,steps-0:35,steps-0:30,steps-0:25,steps-0:20,steps-0:15,steps-0:10,steps-0:05,steps-0:00,cals-5:55,cals-5:50,cals-5:45,cals-5:40,cals-5:35,cals-5:30,cals-5:25,cals-5:20,cals-5:15,cals-5:10,cals-5:05,cals-5:00,cals-4:55,cals-4:50,cals-4:45,cals-4:40,cals-4:35,cals-4:30,cals-4:25,cals-4:20,cals-4:15,cals-4:10,cals-4:05,cals-4:00,cals-3:55,cals-3:50,cals-3:45,cals-3:40,cals-3:35,cals-3:30,cals-3:25,cals-3:20,cals-3:15,cals-3:10,cals-3:05,cals-3:00,cals-2:55,cals-2:50,cals-2:45,cals-2:40,cals-2:35,cals-2:30,cals-2:25,cals-2:20,cals-2:15,cals-2:10,cals-2:05,cals-2:00,cals-1:55,cals-1:50,cals-1:45,cals-1:40,cals-1:35,cals-1:30,cals-1:25,cals-1:20,cals-1:15,cals-1:10,cals-1:05,cals-1:00,cals-0:55,cals-0:50,cals-0:45,cals-0:40,cals-0:35,cals-0:30,cals-0:25,cals-0:20,cals-0:15,cals-0:10,cals-0:05,cals-0:00,activity-5:55,activity-5:50,activity-5:45,activity-5:40,activity-5:35,activity-5:30,activity-5:25,activity-5:20,activity-5:15,activity-5:10,activity-5:05,activity-5:00,a

In [57]:
from autogluon.tabular import TabularDataset, TabularPredictor

import pandas as pd

predictor = TabularPredictor(
    label='bg+1:00',
    eval_metric='rmse',  
    problem_type='regression'  
).fit(
    train_data=train_data,
    num_gpus=1,
    presets='best_quality',
    time_limit=2000,
    verbosity=2,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=3,
    hyperparameters = {'NN_TORCH': {'num_epochs': 10}, 'GBM': {'num_boost_round': 20}},
)

results = predictor.fit_summary()

No path specified. Models will be saved in: "AutogluonModels/ag-20241207_060045"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       28.19 GB / 31.35 GB (89.9%)
Disk Space Avail:   19.19 GB / 19.52 GB (98.3%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fi

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val              eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      LightGBM_BAG_L4  -2.132597  root_mean_squared_error       4.209162  126.755403                1.003675          32.016928            4       True          7
1  WeightedEnsemble_L5  -2.132597  root_mean_squared_error       4.212283  126.873851                0.003122           0.118448            5       True          8
2      LightGBM_BAG_L3  -2.135299  root_mean_squared_error       3.205487   94.738475                0.985597          31.162513            3       True          5
3  WeightedEnsemble_L4  -2.135299  root_mean_squared_error       3.208150   94.744985                0.002663           0.006510            4       True          6
4      LightGBM_BAG_L2  -2.143359  root_mean_squared_error       2.219889   63.575962                1.020283         

In [58]:
predictor.leaderboard(extra_info=True)


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order,num_features,num_models,num_models_w_ancestors,memory_size,memory_size_w_ancestors,memory_size_min,memory_size_min_w_ancestors,num_ancestors,num_descendants,model_type,child_model_type,stopping_metric,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,LightGBM_BAG_L4,-2.132597,root_mean_squared_error,4.209162,126.755403,1.003675,32.016928,4,True,7,365,5,20,448923,1800923,97698,98345,3,1,StackerEnsembleModel,LGBModel,root_mean_squared_error,"{'use_orig_features': True, 'valid_stacker': True, 'max_base_models': 0, 'max_base_models_per_type': 'auto', 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[activity-1:00, bg-3:05, carbs-2:55, carbs-2:25, carbs-0:00, activity-1:15, activity-0:55, bg-0:35, cals-1:50, cals-4:10, activity-0:20, cals-2:10, steps-2:10, carbs-3:00, steps-5:50, steps-1:50, carbs-5:30, carbs-1:15, carbs-0:40, cals-4:40, steps-1:35, activity-3:00, carbs-4:05, cals-4:25, carbs-2:50, bg-2:20, steps-0:30, steps-3:55, cals-5:35, carbs-1:30, cals-1:20, cals-1:30, activity-4:30, bg-2:30, activity-4:55, activity-1:20, activity-2:10, bg-0:30, bg-3:50, steps-0:55, steps-1:30, activity-3:50, carbs-2:45, steps-0:45, cals-4:30, steps-2:20, cals-5:55, cals-2:50, activity-0:25, carbs-3:10, steps-5:25, bg-3:10, steps-4:35, carbs-5:25, bg-5:15, carbs-1:20, cals-3:25, activity-2:30, cals-3:10, activity-5:50, activity-1:50, carbs-1:10, cals-1:10, carbs-1:35, steps-3:20, steps-0:20, steps-1:15, cals-3:05, carbs-4:50, cals-2:40, steps-5:00, activity-2:55, steps-2:35, insulin-5:55, activity-1:30, activity-3:20, carbs-3:40, steps-0:25, bg-0:55, activity-4:25, steps-2:45, cals-4:00, carbs-1:00, steps-3:50, bg-4:45, bg-4:10, bg-4:15, activity-1:25, steps-2:50, steps-0:50, steps-4:00, bg-5:00, carbs-5:20, cals-3:15, activity-3:40, steps-2:25, bg-5:45, cals-3:50, bg-5:30, cals-0:55, ...]",None,"{'learning_rate': 0.05, 'num_boost_round': 20}",{'num_boost_round': 20},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}","[LightGBM_BAG_L1, LightGBM_BAG_L2, LightGBM_BAG_L3]",[WeightedEnsemble_L5]
1,WeightedEnsemble_L5,-2.132597,root_mean_squared_error,4.212283,126.873851,0.003122,0.118448,5,True,8,1,1,21,3091,1804014,3091,98345,4,0,WeightedEnsembleModel,GreedyWeightedEnsembleModel,root_mean_squared_error,"{'use_orig_features': False, 'valid_stacker': True, 'max_base_models': 0, 'max_base_models_per_type': 'auto', 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}",[LightGBM_BAG_L4],None,"{'ensemble_size': 25, 'subsample_size': 1000000}",{'ensemble_size': 1},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwar

In [59]:
predictor.model_best


'WeightedEnsemble_L5'

In [60]:
predictor.model_names()


['LightGBM_BAG_L1',
 'WeightedEnsemble_L2',
 'LightGBM_BAG_L2',
 'WeightedEnsemble_L3',
 'LightGBM_BAG_L3',
 'WeightedEnsemble_L4',
 'LightGBM_BAG_L4',
 'WeightedEnsemble_L5']

In [61]:
# len(filled_train.columns), len(filled_test.columns)

In [62]:
y_pred_auto = predictor.predict(test_data).reset_index().drop('index', axis=1).squeeze()
y_pred_auto

0       8.745707
1       6.852035
2       7.972226
3       9.805777
4       6.609651
          ...   
3639    6.993722
3640    9.805777
3641    7.287155
3642    8.531944
3643    7.254259
Name: bg+1:00, Length: 3644, dtype: float32

In [63]:
cat_cols = train_data.select_dtypes(include=['object', 'category']).columns

num_cols = train_data.select_dtypes(exclude=['object', 'category']).columns.drop('bg+1:00')

print("Categorical Columns:", cat_cols)
print("Numerical Columns:", num_cols)

Categorical Columns: Index(['id', 'p_num', 'time'], dtype='object')
Numerical Columns: Index(['bg-5:55', 'bg-5:50', 'bg-5:45', 'bg-5:40', 'bg-5:35', 'bg-5:30',
       'bg-5:25', 'bg-5:20', 'bg-5:15', 'bg-5:10',
       ...
       'activity-0:45', 'activity-0:40', 'activity-0:35', 'activity-0:30',
       'activity-0:25', 'activity-0:20', 'activity-0:15', 'activity-0:10',
       'activity-0:05', 'activity-0:00'],
      dtype='object', length=432)


In [64]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MinMaxScaler 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [65]:
train_data.shape, test_data.shape

((177024, 436), (3644, 436))

In [66]:
target_column = 'bg+1:00'  
import re

X = train_data.drop(columns=[target_column]).drop(cat_cols, axis=1).rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y = train_data[target_column].rename(index={target_column: 'bg100'})

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

models = {
    'XGBoost': XGBRegressor(device='cuda', objective='reg:squarederror', n_estimators=1000),
    'LightGBM': lgb.LGBMRegressor(device='gpu',n_estimators=1000, verbose= -1),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
}

rmse_dict = {}
predictions = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_dict[name] = rmse
    predictions.append(y_pred)
    print(f"{name} RMSE: {rmse:.4f}")

y_pred_ensemble = np.mean(predictions, axis=0)
rmse_ensemble = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))
print(f"Ensemble RMSE: {rmse_ensemble:.4f}")


# model = lgb.LGBMRegressor(device='gpu', n_estimators=5000, verbose=1)

# model.fit(X_train, y_train)

# y_pred_lgb = model.predict(X_test)

# print('*'*50)
# print('y_pred shape', y_pred_lgb.shape)
# print('y_test shape', y_test.shape)



# rmse = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
# print('-'*50)
# print('rmse', rmse)

XGBoost RMSE: 1.7782


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


LightGBM RMSE: 1.7997
RandomForest RMSE: 1.7799
Ensemble RMSE: 1.7260


In [67]:
test_dropped = test_data.drop(columns=[target_column]).drop(cat_cols, axis=1).rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))


In [68]:
y_pred_submission_xgb = models['XGBoost'].predict(test_dropped)
y_pred_submission_lgb = models['LightGBM'].predict(test_dropped)
y_pred_submission_rf = models['RandomForest'].predict(test_dropped)

y_pred_submission = (y_pred_submission_xgb + y_pred_submission_lgb + y_pred_submission_rf) / 3

In [69]:
np.isnan(y_pred_auto).sum()

0

In [70]:
# autogluon + one model
y_auto_xgb = (y_pred_auto + y_pred_submission_xgb)/2
y_auto_lgb = (y_pred_auto + y_pred_submission_lgb)/2
y_auto_rf = (y_pred_auto + y_pred_submission_rf)/2

# autogluon + two model
y_auto_xgb_lgb = (y_pred_auto + y_pred_submission_xgb + y_pred_submission_lgb)/3
y_auto_xgb_rf = (y_pred_auto + y_pred_submission_xgb + y_pred_submission_rf)/3
y_auto_lgb_rf = (y_pred_auto + y_pred_submission_lgb + y_pred_submission_rf)/3

# autogluon + three model
y_auto_all_models = (y_pred_auto + y_pred_submission_xgb + y_pred_submission_lgb + y_pred_submission_rf)/4


In [71]:
# y_pred = np.exp(y_pred)

sub = pd.read_csv('/kaggle/input/brist1d/sample_submission.csv')
sub1 = sub.copy()
sub1['bg+1:00'] = y_auto_xgb
sub1.to_csv('y_auto_xgb.csv', index=False)

sub2 = sub.copy()
sub2['bg+1:00'] = y_auto_lgb
sub2.to_csv('y_auto_lgb.csv', index=False)

sub3 = sub.copy()
sub3['bg+1:00'] = y_auto_rf
sub3.to_csv('y_auto_rf.csv', index=False)

sub4 = sub.copy()
sub4['bg+1:00'] = y_auto_xgb_lgb
sub4.to_csv('y_auto_xgb_lgb.csv', index=False)

sub5 = sub.copy()
sub5['bg+1:00'] = y_auto_xgb_rf
sub5.to_csv('y_auto_xgb_rf.csv', index=False)

sub6 = sub.copy()
sub6['bg+1:00'] = y_auto_lgb_rf
sub6.to_csv('y_auto_lgb_rf.csv', index=False)

sub7 = sub.copy()
sub7['bg+1:00'] = y_auto_all_models
sub7.to_csv('y_auto_all_models.csv', index=False)

sub7

,id,bg+1:00
0,p01_8459,8.694110
1,p01_8460,7.417960
2,p01_8461,7.283696
3,p01_8462,10.143602
4,p01_8463,6.579710
...,...,...
3639,p24_256,7.131062
3640,p24_257,9.424732
3641,p24_258,6.976977
3642,p24_259,8.647396


In [72]:
sub1.shape , len(y_auto_xgb)

((3644, 2), 3644)

In [73]:
# y_pred1 = predictor.predict(filled_test, model='NeuralNetTorch_BAG_L1')
# y_pred2 = predictor.predict(filled_test, model='NeuralNetTorch_BAG_L2')
# y_pred3 = predictor.predict(filled_test, model='NeuralNetTorch_BAG_L3')
# y_pred4 = predictor.predict(filled_test, model='LightGBM_BAG_L1')
# y_pred5 = predictor.predict(filled_test, model='LightGBM_BAG_L2')
# y_pred6 = predictor.predict(filled_test, model='LightGBM_BAG_L3')
# y_pred7 = predictor.predict(filled_test, model='LightGBM_BAG_L4')
# y_pred8 = predictor.predict(filled_test, model='WeightedEnsemble_L2')
# y_pred9 = predictor.predict(filled_test, model='WeightedEnsemble_L3')
# y_pred10 = predictor.predict(filled_test, model='WeightedEnsemble_L4')
# y_pred11 = predictor.predict(filled_test, model='WeightedEnsemble_L5')

# sample_submission = pd.read_csv('/kaggle/input/brist1d/sample_submission.csv')

# sub1 = sample_submission.copy()
# sub1['bg+1:00'] = y_pred1
# sub1.to_csv('NeuralNetTorch_BAG_L1.csv', index=False)

# sub2 = sample_submission.copy()
# sub2['bg+1:00'] = y_pred2
# sub2.to_csv('NeuralNetTorch_BAG_L2.csv', index=False)

# sub3 = sample_submission.copy()
# sub3['bg+1:00'] = y_pred3
# sub3.to_csv('NeuralNetTorch_BAG_L3.csv', index=False)

# sub4 = sample_submission.copy()
# sub4['bg+1:00'] = y_pred4
# sub4.to_csv('LightGBM_BAG_1L1.csv', index=False)

# sub5 = sample_submission.copy()
# sub5['bg+1:00'] = y_pred5
# sub5.to_csv('LightGBM_BAG_L2.csv', index=False)

# sub6 = sample_submission.copy()
# sub6['bg+1:00'] = y_pred6
# sub6.to_csv('LightGBM_BAG_L3.csv', index=False)

# sub7 = sample_submission.copy()
# sub7['bg+1:00'] = y_pred7
# sub7.to_csv('LightGBM_BAG_L4.csv', index=False)

# sub8 = sample_submission.copy()
# sub8['bg+1:00'] = y_pred8
# sub8.to_csv('WeightedEnsemble_L2.csv', index=False)

# sub9 = sample_submission.copy()
# sub9['bg+1:00'] = y_pred9
# sub9.to_csv('WeightedEnsemble_L3.csv', index=False)

# sub10 = sample_submission.copy()
# sub10['bg+1:00'] = y_pred10
# sub10.to_csv('WeightedEnsemble_L4.csv', index=False)

# sub11 = sample_submission.copy()
# sub11['bg+1:00'] = y_pred11
# sub11.to_csv('WeightedEnsemble_L5.csv', index=False)

In [74]:
# sub1 = pd.read_csv('NeuralNetTorch_BAG_L1.csv')
# sub2 = pd.read_csv('NeuralNetTorch_BAG_L2.csv')
# sub3 = pd.read_csv('NeuralNetTorch_BAG_L3.csv')
# sub4 = pd.read_csv('LightGBM_BAG_1L1.csv')
# sub5 = pd.read_csv('LightGBM_BAG_L2.csv')
# sub6 = pd.read_csv('LightGBM_BAG_L3.csv')
# sub7 = pd.read_csv('LightGBM_BAG_L4.csv')
# sub8 = pd.read_csv('WeightedEnsemble_L2.csv')
# sub9 = pd.read_csv('WeightedEnsemble_L3.csv')
# sub10 = pd.read_csv('WeightedEnsemble_L4.csv')
# sub11 = pd.read_csv('WeightedEnsemble_L5.csv')


# predictions = [
#     sub1['bg+1:00'],
#     sub2['bg+1:00'],
#     sub3['bg+1:00'],
#     sub4['bg+1:00'],
#     sub5['bg+1:00'],
#     sub6['bg+1:00'],
#     sub7['bg+1:00'],
#     sub8['bg+1:00'],
#     sub9['bg+1:00'],
#     sub10['bg+1:00'],
#     sub11['bg+1:00']
# ]

# average_predictions = sum(predictions) / len(predictions)

# final_submission = sub1.copy()
# final_submission['bg+1:00'] = average_predictions

# final_submission.to_csv('bagging_ensemble_submission.csv', index=False)
# print("Bagging ensemble created and saved as 'bagging_ensemble_submission.csv'")

In [75]:
# predictions_stack = np.column_stack(predictions)

# median_predictions = np.median(predictions_stack, axis=1)

# final_submission = sub1.copy()
# final_submission['bg+1:00'] = median_predictions

# final_submission.to_csv('voting_ensemble_submission.csv', index=False)
# print("Voting ensemble created and saved as 'voting_ensemble_submission.csv'")

In [76]:
# target_column = 'bg+1:00'  
# import re
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MinMaxScaler 
# import lightgbm as lgb
# from sklearn.metrics import mean_squared_error

# X = train.drop(columns=[target_column]).drop(cat_cols, axis=1).rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
# y = train[target_column].rename(index={target_column: 'bg100'})

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# model = lgb.LGBMRegressor(device='gpu', n_estimators=1000, verbose=1)

# model.fit(X_train, y_train)

# y_pred_lgb = model.predict(X_test)

# print('*'*50)
# print('y_pred has NaN', np.isnan(y_pred).sum())
# print('y_test has NaN', np.isnan(y_test).sum())



# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print('-'*50)
# print('rmse', rmse)